In [1]:
###########################################################
#                                                         #
#               TRAITEMENT SIMUS FORTRAN                  #
#                     STATISTIQUES                        #
#                                                         #
#                         avec                            #
#                   - lecture arborescence                #
#                  analyse                                #
#                   - Delta Fluo                          #
#                   - Delta Energy GMol                   #
#                   - R/L Ca+                             #
#                   - distance inter ionique              #
#                   - trajectoire GMol                    #
#                                                         #
#                          02/2020     Adrien Poindron    #
###########################################################

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

# Imports

In [3]:
# %pylab

import numpy
import matplotlib
from matplotlib import pylab, mlab, pyplot
from mpl_toolkits.mplot3d import Axes3D
np = numpy
plt = pyplot

matplotlib.rcParams.update({'font.size': 21})
# matplotlib.rcParams["figure.dpi"] = 300

from IPython.display import display
from IPython.core.pylabtools import figsize, getfigs

from pylab import *
from numpy import *

from scipy.optimize import curve_fit
from scipy.signal import savgol_filter, butter

In [4]:
import os
from os import listdir
from os.path import isfile, join

import tkinter as tk
from tkinter import filedialog
%matplotlib qt

from pathlib import Path
from matplotlib import gridspec
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

In [5]:
from natsort import natsorted   # natural sort marche de ouf
import re                       # find '/' in strings
import timeit
cm = pylab.get_cmap('summer')   # colormap par défaut color=cm(k/len(array))

In [23]:
# Fonctions persos de Jofre et Adrien

# %run function_jofre.ipy
%run ../../Functions/1.2/data_fct_Adrien_point_by_point

In [7]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+

Omega = 2.05e6*2*pi # 2.047e6

bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          # écrantage piège réel GiantMol
wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


In [74]:
def load_Temp_init_bin_Lan(str_load, flag_print):      

    fid = open(str_load+'.bin',  'rb')   

    a    = fromfile(fid,  int32, 1)        # Read record start tag   
    aux  = fromfile(fid,  int32, 1)   
    junk = fromfile(fid,  int32, 1)        # Read record stop tag   
    N_ions = aux[0]   

    b    = fromfile(fid,  int32, 1)        # Read record start tag   
    aux  = fromfile(fid,  int32, 1)   
    junk = fromfile(fid,  int32, 1)        # Read record stop tag   
    j_save = aux[0]   

    c    = fromfile(fid,  int32, 1)        # Read record start tag   
    aux  = fromfile(fid,  float64, 1)   
    junk = fromfile(fid,  int32, 1)        # Read record stop tag   
    dt_j_save_next = aux[0]   

    d    = fromfile(fid,  int32, 1)        # Read record start tag   
    aux  = fromfile(fid,  float64, 1)   
    junk = fromfile(fid,  int32, 1)        # Read record stop tag   
    eta = aux[0]   

    e    = fromfile(fid,  int32,   1   )        # Read record start tag   
    Temp = fromfile(fid,  float64, 1)   
    junk = fromfile(fid,  int32,   1   )        # Read record stop tag   

    f    = fromfile(fid,  int32,   1   )        # Read record start tag   
#    junk = fromfile(fid,  int32,   1   )        # Read record start tag   
    print('len save_T', junk)   
    save_T = fromfile(fid,  float64, 7*j_save)   
    junk = fromfile(fid,  int32,   1   )        # Read record stop tag   

    fid.close   

    if flag_print == 1:
        print(a, b, c, d, e, f)   
        print('N_ions', N_ions)   
        print('j_save', j_save)   
        print('dt_j_save_next', dt_j_save_next)   
        print('eta', eta)   
        print('Temp', Temp)   


    save_T = reshape(save_T, (j_save, 7), order='F')   
    print(save_T[0])   

    return N_ions, j_save, dt_j_save_next, eta, Temp, save_T 

# Sélection fichier

In [10]:
## GUI for data loading
# Select one data file all the way down to the directories
# Temp_SimuType0_N01024_Vrf0064_Udc0.5000D+00V_D1.0_S1.0RFG.dat

file_cfg, slash_cfg, all_subdir = load_gui(filter_nocomplete=1)

file_path = file_cfg[0]
dir_path = file_cfg[1]
work_rep = file_cfg[2]
filename = file_cfg[3]

myslashpos = slash_cfg[0]
slashcond = slash_cfg[1]

> Répertoire : D:/CIMLuser/adrien/Simulations/20200710/DC03_RF09/Try00
> Filename : Temp_SimuType0_N01024_Vrf0059_Udc0.2000D+01V_D1.0_S1.0RFG.dat
> myslashpos | [2, 11, 18, 30, 39, 49]
> slashcond | -2
Points deleted because they were not complete []   0 pt(s)
Total number of data directories 78


In [11]:
## Condition for data
points_and_coord, condition_parameters = simu_conditions(all_subdir, myslashpos, slashcond, filename)

> condition names ['DC', 'RF']
> number of points 13
000 > DC03_RF09
001 > DC03_RF10
002 > DC03_RF11
003 > DC05_RF07
004 > DC05_RF09
005 > DC05_RF10
006 > DC05_RF11
007 > DC07_RF09
008 > DC07_RF10
009 > DC07_RF11
010 > DC09_RF09
011 > DC09_RF10
012 > DC09_RF11
> N_ions = 1024
> e_GMol = 50


In [12]:
## Data loading (1D arrays)
data_name, num_runs, PMandT, Gmol_data, r_LC_clip, dim_nu \
= data_retrieve(all_subdir, points_and_coord,condition_parameters,slash_cfg) # stopping_power

data0 = data_name[0]
data2 = data_name[1]
data4 = data_name[2]

PMvar = PMandT[0]
Tvar = PMandT[1]

deltaEc = Gmol_data[0]
deltaEcRel = Gmol_data[1]
t_c = Gmol_data[2]

> Points | 13
> Simulations pour chaque point | ['Try00', 'Try01', 'Try02', 'Try03', 'Try04', 'Try05']
Hello
Point n° 0
00 - 00  >  D:/CIMLuser/adrien/Simulations/20200710\DC03_RF09\Try00/Temp_SimuType0_N01024_Vrf0059_Udc0.2000D+01V_D1.0_S1.0RFG
00 - 01  >  D:/CIMLuser/adrien/Simulations/20200710\DC03_RF09\Try01/Temp_SimuType0_N01024_Vrf0059_Udc0.2000D+01V_D1.0_S1.0RFG
00 - 02  >  D:/CIMLuser/adrien/Simulations/20200710\DC03_RF09\Try02/Temp_SimuType0_N01024_Vrf0059_Udc0.2000D+01V_D1.0_S1.0RFG
00 - 03  >  D:/CIMLuser/adrien/Simulations/20200710\DC03_RF09\Try03/Temp_SimuType0_N01024_Vrf0059_Udc0.2000D+01V_D1.0_S1.0RFG
00 - 04  >  D:/CIMLuser/adrien/Simulations/20200710\DC03_RF09\Try04/Temp_SimuType0_N01024_Vrf0059_Udc0.2000D+01V_D1.0_S1.0RFG
00 - 05  >  D:/CIMLuser/adrien/Simulations/20200710\DC03_RF09\Try05/Temp_SimuType0_N01024_Vrf0059_Udc0.2000D+01V_D1.0_S1.0RFG
Point n° 1
01 - 00  >  D:/CIMLuser/adrien/Simulations/20200710\DC03_RF10\Try00/Temp_SimuType0_N01024_Vrf0064_Udc0.2000D+01V_

10 - 04  >  D:/CIMLuser/adrien/Simulations/20200710\DC09_RF09\Try04/Temp_SimuType0_N01024_Vrf0059_Udc0.5000D+01V_D1.0_S1.0RFG
10 - 05  >  D:/CIMLuser/adrien/Simulations/20200710\DC09_RF09\Try05/Temp_SimuType0_N01024_Vrf0059_Udc0.5000D+01V_D1.0_S1.0RFG
Point n° 11
11 - 00  >  D:/CIMLuser/adrien/Simulations/20200710\DC09_RF10\Try00/Temp_SimuType0_N01024_Vrf0064_Udc0.5000D+01V_D1.0_S1.0RFG
11 - 01  >  D:/CIMLuser/adrien/Simulations/20200710\DC09_RF10\Try01/Temp_SimuType0_N01024_Vrf0064_Udc0.5000D+01V_D1.0_S1.0RFG
11 - 02  >  D:/CIMLuser/adrien/Simulations/20200710\DC09_RF10\Try02/Temp_SimuType0_N01024_Vrf0064_Udc0.5000D+01V_D1.0_S1.0RFG
11 - 03  >  D:/CIMLuser/adrien/Simulations/20200710\DC09_RF10\Try03/Temp_SimuType0_N01024_Vrf0064_Udc0.5000D+01V_D1.0_S1.0RFG
11 - 04  >  D:/CIMLuser/adrien/Simulations/20200710\DC09_RF10\Try04/Temp_SimuType0_N01024_Vrf0064_Udc0.5000D+01V_D1.0_S1.0RFG
11 - 05  >  D:/CIMLuser/adrien/Simulations/20200710\DC09_RF10\Try05/Temp_SimuType0_N01024_Vrf0064_Udc0.500

In [117]:
# Where SNR for each try in each  are compared to a threshold
# If SNR > some_threshold , we count it as a detection
# Then for each point the ratio number_of_detection/number_of_try is calculated
# This detection efficiency 

SNR_trig_threshold = 1
shapevar = ((len(points_and_coord), len(num_runs)))
trig_detection = np.zeros((shapevar))
for i in range(len(points_and_coord)):
    SNR = Gmol_data[2][i]
    for k,m in enumerate(SNR):
        if m > SNR_trig_threshold:
            trig_detection[i,k] = 1

print(mean(trig_detection,axis=1))

print(len(trig_detection))

[0.83333333 0.         0.         0.         0.33333333 0.
 0.         0.         0.         0.         0.         0.
 0.        ]
13


(13, 6)

In [14]:
# Where is written Detuning and how much it is
temp_pos = filename.find('0RFG')
Detuning = filename[temp_pos-7:temp_pos-4]
print(Detuning)

1.0


# Plots
Nuage d'ions, Temp et Pos

## Load temperature from Langevin files

In [17]:
## Data loading with hardcode names

plot_point = 0
plot_try = 3
dirname_hc = str(data0[plot_point][plot_try])+'/'

datainit_hc = 'Temp_3D_Harmo_N1024_T500uK_F0.15D-19Kg_s_5'
data0_hc    = '/Temp_SimuType0_00_N01024_Vrf0070_Udc0.5000D+01V_D1.0_S1.0RFG'
data4_hc    = '/Temp_SimuType4_00_N01024_Vrf0070_Udc0.5000D+01V_D1.0_S1.0RFG'
data2_hc    = '/Temp_SimuType2_00_N01024_Vrf0070_Udc0.5000D+01V_D1.0_S1.0RFG'

print(dirname_hc)
print(datainit_hc)

D:/CIMLuser/adrien/Simulations/20200710\DC03_RF09\Try03/
Temp_3D_Harmo_N1024_T500uK_F0.15D-19Kg_s_5


In [75]:
N_ions, j_save, dt_j_save_next, eta, Temp, save_T =\
load_Temp_init_bin_Lan(dirname_hc+datainit_hc,1)

#print('N_ions',N_ions)
#print('j_save',j_save)
#print('dt*j_save_next',dt_j_save_next)
#print('eta',eta)
#print('Temp',Temp)
#print('save_T',save_T)
#print('shape(save_T)',shape(save_T))

#  Sacahnt que dans le Fortran
# dt   = 2d-9
# j_save_next  = 100
# eta = 1.5d-21
# Temperature = 0.5d-3

len save_T [8]
[4] [4] [8] [8] [8] [110880]
N_ions 1024
j_save 1980
dt_j_save_next 5e-07
eta 1.5e-20
Temp [0.0005]
[5.05000000e-07 3.97240563e-04 1.38548544e-03 1.01171426e-06
 1.78835031e+01 4.13538231e+00 7.87416722e+01]


In [85]:
# Get T and fluo PM
onlyfiles = [f for f in listdir(dirname_hc) if isfile(join(dirname_hc, f)) and "SimuType" in f and ".dat" in f]
ta, te, ti = plot_T_and_PM_Init_Inje_Evol(dirname_hc,sort(onlyfiles)[0].strip('.dat')[4:],0,'coucou' )

# ,xlim1=(3.458,3.582),ylim1=(2e-4,50),ylim2=(19.5,65)

In [97]:
n_dt = 100

i_free__fly_Lan = 2*1e3
i_free__fly = 100
i_therm_fly = 2000
i_laser_fly = 1000*3
i_cool__fly = 2000

dt = 1/2e6

In [98]:
# Plot temperature Langevin + md laser

tt_Lan = linspace(0,dt_j_save_next*j_save,j_save) # dt_j_save_next*100 duration of Langevin run  ,
tt_md = ta[0]
T_aux = te[0]

tt = concatenate((tt_Lan,tt_md))
T_all = concatenate((save_T[:,4:],T_aux),axis=0)

PM = ti[0]

#fig_name = file_name[-9:] 
figure('Temperature'); clf()
ax1 = subplot(211)
ax1.semilogy(tt*1e3,T_all[:,0], label='Tx',color='C0')
ax1.semilogy(tt*1e3,T_all[:,1], label='Ty',color='C1')
ax1.semilogy(tt*1e3,T_all[:,2], label='Tz',color='C2')
#semilogy(tt_Lan*1.e3,save_T[:,0], label='Tx',color='C0')
#semilogy(tt_Lan*1.e3,save_T[:,1], label='Ty',color='C1')
#semilogy(tt_Lan*1.e3,save_T[:,2], label='Tz',color='C2')
ax1.vlines(ta[1]*1e3,1e-4,1e4,color='red')
ax1.vlines(ta[2]*1e3,1e-4,1e4,color='red')
time_switch = i_free__fly_Lan*dt
ax1.vlines(time_switch*1e3,1e-4,1e4,color='blue')
time_switch = (i_free__fly_Lan+i_laser_fly)*dt
ax1.vlines(time_switch*1e3,1e-4,1e4,color='orange')
time_switch = (i_free__fly_Lan+2*i_laser_fly)*dt
ax1.vlines((time_switch+ta[2]-ta[1])*1e3,1e-4,1e4,color='purple')
ax1.grid()
# annotate('Laser ON', xy=(0.5,350), xycoords='data',
# size=24, ha='left', va='top', color='xkcd:azul',
# bbox=dict(boxstyle='round', fc='white',edgecolor='xkcd:azul')
,
# ~ xlabel('time[ms]')
# ~ ylabel('T[K]')
plt.setp(ax1.get_xticklabels(),visible=False)

ax2 = subplot(212,sharex=ax1)
plot(tt_md*1e3,PM, label='Tx')
time_switch = i_free__fly_Lan*dt
ax2.vlines(time_switch*1e3,1e-4,75,color='blue')
time_switch = (i_free__fly_Lan+i_laser_fly)*dt
ax2.vlines(time_switch*1e3,1e-4,75,color='orange')
time_switch = (i_free__fly_Lan+2*i_laser_fly)*dt
ax2.vlines((time_switch+ta[2]-ta[1])*1e3,1e-4,75,color='purple')
ax2.grid()
# annotate('laser off après injection', xy=(0.5,350), xycoords='data',
# size=24, ha='left', va='top', color='xkcd:azul',
# bbox=dict(boxstyle='round', fc='white',edgecolor='xkcd:azul'))

xlabel('time[ms]')
ylabel('Counts')

plt.tight_layout()
subplots_adjust(hspace=0.015)

In [112]:
list(dict.fromkeys(points_and_coord))

['DC03_RF09',
 'DC03_RF10',
 'DC03_RF11',
 'DC05_RF07',
 'DC05_RF09',
 'DC05_RF10',
 'DC05_RF11',
 'DC07_RF09',
 'DC07_RF10',
 'DC07_RF11',
 'DC09_RF09',
 'DC09_RF10',
 'DC09_RF11']

In [111]:
# Plot all temp rf relax après refroidissement\n"
for patate,ketchup in enumerate(list(dict.fromkeys(points_and_coord))):

    plot_point = patate
    fname = 'Temp_alltry_'+ketchup


    fig, ax = plt.subplots(3, 2,num=fname,clear=True,sharex=True,sharey=True)
    fig.set_size_inches(11.69,8.27)
    # suptitle('Saturation 866')

    to_plot = 0
    for l in range(0,2):
        for k in range(0,3):

            dirname_hc = str(data0[plot_point][to_plot])+'/'
            N_ions, j_save, dt_j_save_next, eta, Temp, save_T = \
            load_Temp_init_bin_Lan(dirname_hc+datainit_hc,flag_print=0)
            # Get T and fluo PM
            onlyfiles = [f for f in listdir(dirname_hc) if isfile(join(dirname_hc, f)) and 'SimuType' in f and '.dat' in f]
            ta, te, ti = plot_T_and_PM_Init_Inje_Evol(dirname_hc,sort(onlyfiles)[0].strip('.dat')[4:],0,'coucou' )

            tt_Lan = linspace(0,dt_j_save_next*j_save,j_save) # dt_j_save_next*100 duration of Langevin run 
            tt_md = ta[0]
            T_aux = te[0]

            tt = concatenate((tt_Lan,tt_md))
            T_all = concatenate((save_T[:,4:],T_aux),axis=0)

            PM = ti[0]

            ax[k,l].semilogy(tt*1e3,T_all[:,0], label='Tx',color='C0')
            ax[k,l].semilogy(tt*1e3,T_all[:,1], label='Ty',color='C1')
            ax[k,l].semilogy(tt*1e3,T_all[:,2], label='Tz',color='C2')
            ax[k,l].vlines(ta[1]*1e3,1e-4,1e4,color='red')
            ax[k,l].vlines(ta[2]*1e3,1e-4,1e4,color='red')
            time_switch = i_free__fly_Lan*dt
            ax[k,l].vlines(time_switch*1e3,1e-4,1e4,color='blue')
            time_switch = (i_free__fly_Lan+i_laser_fly)*dt
            ax[k,l].vlines(time_switch*1e3,1e-4,1e4,color='orange')
            time_switch = (i_free__fly_Lan+2*i_laser_fly)*dt
            ax[k,l].vlines((time_switch+ta[2]-ta[1])*1e3,1e-4,1e4,color='purple')
            ax[k,l].set_yticks([1e-4,1e-2,1,1e2,1e4])
            ax[k,l].grid()
            #     ax[k].set_ylim(-0.1,0.55)
            ax[k,l].xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.5))
            ax[k,l].annotate('try'+str(to_plot), xy=(0.80, 0.99), xycoords='axes fraction', # pts_concerned[to_plot]
            size=12, ha='left', va='top',
            bbox=dict(boxstyle='round', fc='w'))
            to_plot += 1
            if to_plot == 6:
                break
        if to_plot == 6:
            break

    # ax[3].set_xlim(-10,4000)
    # ax[k,l].set_ylim(2e-4,1e-1)
    fig.suptitle(list(dict.fromkeys(points_and_coord))[plot_point])
    fig.text(0.56, 0.04, 'time [ms]', ha='center')
    fig.text(0.04, 0.54, 'T [K]', va='center', rotation='vertical')

    # fig.suptitle(pt_to_plot[0][:-6],fontsize=10)

    # fig.text(0.5, 0.01, 'f', ha='center')
    # fig.text(0.01, 0.5, r'$| FFT(HC(t)) | / N_{sample}$', va='center', rotation='vertical')

    fig.set_size_inches(11.69,8.27)
    plt.tight_layout()
    subplots_adjust(hspace=0.05, wspace = 0.05, bottom = 0.120, left = 0.140, top = 0.92) # hspace=0, top=0.95, left = 0.065, bottom = 0.07
    savefig(fname+'.png') 

C:\Anaconda3\envs\py37\lib\site-packages\matplotlib\axes\_base.py:3477: UserWarning: Attempted to set non-positive ylimits for log-scale axis; invalid limits will be ignored.
  'Attempted to set non-positive ylimits for log-scale axis; '


len save_T [8]
[5.05000000e-07 1.01548937e-03 6.71730325e-02 2.29217981e-06
 1.79474787e+01 4.37648391e+00 7.91182553e+01]
len save_T [8]
[5.05000000e-07 6.92704091e-04 1.01630676e-02 3.22024709e-06
 1.69195590e+01 3.90546636e+00 7.46999518e+01]
len save_T [8]
[5.05000000e-07 5.72896435e-05 2.57597598e-03 7.79515994e-06
 1.74460290e+01 4.46020910e+00 7.61256628e+01]
len save_T [8]
[5.05000000e-07 9.55335681e-04 1.08380207e-02 3.65433681e-06
 1.73907886e+01 3.81140621e+00 7.80533896e+01]
len save_T [8]
[5.05000000e-07 2.69208421e-03 2.23136037e-02 1.50363249e-07
 1.78736034e+01 4.84998481e+00 7.87458192e+01]
len save_T [8]
[5.05000000e-07 3.97240563e-04 1.38548544e-03 1.01171426e-06
 1.78835031e+01 4.13538231e+00 7.87416722e+01]
len save_T [8]
[5.05000000e-07 3.87877796e-04 6.02442871e-02 1.42633628e-06
 1.36843834e+01 5.92321188e+00 7.88504632e+01]
len save_T [8]
[5.05000000e-07 2.48954624e-05 8.69220869e-03 6.59146693e-06
 1.35160376e+01 5.51223608e+00 7.67137699e+01]
len save_T [8]
[

len save_T [8]
[5.05000000e-07 8.92588152e-04 1.15718821e-03 1.02948275e-06
 1.48784830e+01 5.25826737e+00 7.11288642e+01]
len save_T [8]
[5.05000000e-07 4.07759820e-03 1.77734824e-02 7.37787807e-05
 1.54945551e+01 5.34301113e+00 7.44501612e+01]
len save_T [8]
[5.05000000e-07 1.35760629e-05 2.01462555e-02 6.42497017e-05
 1.54988366e+01 4.87600840e+00 7.30754205e+01]
len save_T [8]
[5.05000000e-07 1.15495568e-04 2.24007334e-04 9.94431592e-09
 1.49433445e+01 5.39549511e+00 7.25586169e+01]
len save_T [8]
[5.05000000e-07 2.35747930e-03 1.16285809e-02 1.20118718e-05
 1.38528132e+01 6.12470172e+00 6.70800724e+01]
len save_T [8]
[5.05000000e-07 3.38533952e-04 3.45366299e-02 4.04563722e-05
 1.15742225e+01 5.88118496e+00 7.46115168e+01]
len save_T [8]
[5.05000000e-07 6.92863709e-03 5.09497527e-02 8.87141236e-05
 1.15052369e+01 7.58274888e+00 7.52410455e+01]
len save_T [8]
[5.05000000e-07 3.02351773e-04 1.57266124e-02 7.80664941e-07
 1.12749995e+01 6.17441391e+00 7.27064548e+01]
len save_T [8]
[

In [113]:
# Plot all fluo rf relax après refroidissement

for patate,ketchup in enumerate(list(dict.fromkeys(points_and_coord))):

    plot_point = patate
    fname = 'Fluo_alltry_'+ketchup

    fig, ax = plt.subplots(3, 2,num=fname,clear=True,sharex=True,sharey=True)
    fig.set_size_inches(11.69,8.27)
    # suptitle('Saturation 866')


    to_plot = 0
    for l in range(0,2):
        for k in range(0,3):

            dirname_hc = str(data0[plot_point][to_plot])+'/'
            N_ions, j_save, dt_j_save_next, eta, Temp, save_T =\
            load_Temp_init_bin_Lan(dirname_hc+datainit_hc,0)
            # Get T and fluo PM
            onlyfiles = [f for f in listdir(dirname_hc) if isfile(join(dirname_hc, f)) and 'SimuType' in f and '.dat' in f]
            ta, te, ti = plot_T_and_PM_Init_Inje_Evol(dirname_hc,sort(onlyfiles)[0].strip('.dat')[4:],0,'coucou' )

            tt_Lan = linspace(0,dt_j_save_next*j_save,j_save) # dt_j_save_next*100 duration of Langevin run
            tt_md = ta[0]
            T_aux = te[0]

            tt = concatenate((tt_Lan,tt_md))
            T_all = concatenate((save_T[:,4:],T_aux),axis=0)

            PM = ti[0]

            ax[k,l].plot(tt_md*1e3,PM, label='Tx',color='C0')
            ax[k,l].vlines(ta[1]*1e3,1e-4,75,color='red')
            ax[k,l].vlines(ta[2]*1e3,1e-4,75,color='red')
            time_switch = i_free__fly_Lan*dt
            ax[k,l].vlines(time_switch*1e3,1e-4,75,color='blue')
            time_switch = (i_free__fly_Lan+i_laser_fly)*dt
            ax[k,l].vlines(time_switch*1e3,1e-4,75,color='orange')
            time_switch = (i_free__fly_Lan+2*i_laser_fly)*dt
            ax[k,l].vlines((time_switch+ta[2]-ta[1])*1e3,1e-4,75,color='purple')
            ax[k,l].grid()
        #     ax[k].set_ylim(-0.1,0.55)
            ax[k,l].xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.5))
            ax[k,l].annotate('try'+str(to_plot), xy=(0.80, 0.99), xycoords='axes fraction', # pts_concerned[to_plot]
                    size=12, ha='left', va='top',
                    bbox=dict(boxstyle='round', fc='w'))
            to_plot += 1
            if to_plot == 6:
                break
        if to_plot == 6:
            break
    # ax[3].set_xlim(-10,4000),
    # ax[k,l].set_ylim(2e-4,1e-1),
    fig.suptitle(ketchup)
    fig.text(0.56, 0.04, 'time [ms]', ha='center')
    fig.text(0.04, 0.54, 'T [K]', va='center', rotation='vertical')

    # fig.suptitle(pt_to_plot[0][:-6],fontsize=10)

    # fig.text(0.5, 0.01, 'f', ha='center')
    # fig.text(0.01, 0.5, r'$| FFT(HC(t)) | / N_{sample}$', va='center', rotation='vertical')

    fig.set_size_inches(11.69,8.27)
    plt.tight_layout()
    subplots_adjust(hspace=0.05, wspace = 0.05, bottom = 0.120, left = 0.140, top = 0.92) # hspace=0, top=0.95, left = 0.065, bottom = 0.07
    savefig(fname+'.png')

len save_T [8]
[5.05000000e-07 1.01548937e-03 6.71730325e-02 2.29217981e-06
 1.79474787e+01 4.37648391e+00 7.91182553e+01]
len save_T [8]
[5.05000000e-07 6.92704091e-04 1.01630676e-02 3.22024709e-06
 1.69195590e+01 3.90546636e+00 7.46999518e+01]
len save_T [8]
[5.05000000e-07 5.72896435e-05 2.57597598e-03 7.79515994e-06
 1.74460290e+01 4.46020910e+00 7.61256628e+01]
len save_T [8]
[5.05000000e-07 9.55335681e-04 1.08380207e-02 3.65433681e-06
 1.73907886e+01 3.81140621e+00 7.80533896e+01]
len save_T [8]
[5.05000000e-07 2.69208421e-03 2.23136037e-02 1.50363249e-07
 1.78736034e+01 4.84998481e+00 7.87458192e+01]
len save_T [8]
[5.05000000e-07 3.97240563e-04 1.38548544e-03 1.01171426e-06
 1.78835031e+01 4.13538231e+00 7.87416722e+01]
len save_T [8]
[5.05000000e-07 3.87877796e-04 6.02442871e-02 1.42633628e-06
 1.36843834e+01 5.92321188e+00 7.88504632e+01]
len save_T [8]
[5.05000000e-07 2.48954624e-05 8.69220869e-03 6.59146693e-06
 1.35160376e+01 5.51223608e+00 7.67137699e+01]
len save_T [8]
[

[5.05000000e-07 8.92588152e-04 1.15718821e-03 1.02948275e-06
 1.48784830e+01 5.25826737e+00 7.11288642e+01]
len save_T [8]
[5.05000000e-07 4.07759820e-03 1.77734824e-02 7.37787807e-05
 1.54945551e+01 5.34301113e+00 7.44501612e+01]
len save_T [8]
[5.05000000e-07 1.35760629e-05 2.01462555e-02 6.42497017e-05
 1.54988366e+01 4.87600840e+00 7.30754205e+01]
len save_T [8]
[5.05000000e-07 1.15495568e-04 2.24007334e-04 9.94431592e-09
 1.49433445e+01 5.39549511e+00 7.25586169e+01]
len save_T [8]
[5.05000000e-07 2.35747930e-03 1.16285809e-02 1.20118718e-05
 1.38528132e+01 6.12470172e+00 6.70800724e+01]
len save_T [8]
[5.05000000e-07 3.38533952e-04 3.45366299e-02 4.04563722e-05
 1.15742225e+01 5.88118496e+00 7.46115168e+01]
len save_T [8]
[5.05000000e-07 6.92863709e-03 5.09497527e-02 8.87141236e-05
 1.15052369e+01 7.58274888e+00 7.52410455e+01]
len save_T [8]
[5.05000000e-07 3.02351773e-04 1.57266124e-02 7.80664941e-07
 1.12749995e+01 6.17441391e+00 7.27064548e+01]
len save_T [8]
[5.05000000e-07 

In [115]:
for g,h in enumerate(points_and_coord):
    print(g,h)

0 DC03_RF09
1 DC03_RF10
2 DC03_RF11
3 DC05_RF07
4 DC05_RF09
5 DC05_RF10
6 DC05_RF11
7 DC07_RF09
8 DC07_RF10
9 DC07_RF11
10 DC09_RF09
11 DC09_RF10
12 DC09_RF11


In [140]:
Gmol_data[2]

detect_rate = []
detect_threshold = 1

for k in range(shape(Gmol_data[2])[0]):
    print('k',k)
    detect_rate.append(0)
    for l,m in enumerate(Gmol_data[2][k]):
        print('... l',l)
        if m > detect_threshold:
            detect_rate[k] += 1
    detect_rate[k] = detect_rate[k] / shape(Gmol_data[2])[1]
    
for n,o in enumerate(list(dict.fromkeys(points_and_coord))):
    print(o)
    print(detect_rate[n])

k 0
... l 0
... l 1
... l 2
... l 3
... l 4
... l 5
k 1
... l 0
... l 1
... l 2
... l 3
... l 4
... l 5
k 2
... l 0
... l 1
... l 2
... l 3
... l 4
... l 5
k 3
... l 0
... l 1
... l 2
... l 3
... l 4
... l 5
k 4
... l 0
... l 1
... l 2
... l 3
... l 4
... l 5
k 5
... l 0
... l 1
... l 2
... l 3
... l 4
... l 5
k 6
... l 0
... l 1
... l 2
... l 3
... l 4
... l 5
k 7
... l 0
... l 1
... l 2
... l 3
... l 4
... l 5
k 8
... l 0
... l 1
... l 2
... l 3
... l 4
... l 5
k 9
... l 0
... l 1
... l 2
... l 3
... l 4
... l 5
k 10
... l 0
... l 1
... l 2
... l 3
... l 4
... l 5
k 11
... l 0
... l 1
... l 2
... l 3
... l 4
... l 5
k 12
... l 0
... l 1
... l 2
... l 3
... l 4
... l 5
DC03_RF09
0.8333333333333334
DC03_RF10
0.0
DC03_RF11
0.0
DC05_RF07
0.0
DC05_RF09
0.3333333333333333
DC05_RF10
0.0
DC05_RF11
0.0
DC07_RF09
0.0
DC07_RF10
0.0
DC07_RF11
0.0
DC09_RF09
0.0
DC09_RF10
0.0
DC09_RF11
0.0


In [135]:
detect_rate[0]

[]

In [142]:
Udc = [0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 6, 7, 7.5, 3.33, 3.66, 4.5, 5.33, 5.66, 6, 6.5]
Urf = [10.77, 16.15, 21.54, 26.92, 32.31, 37.69, 43.08, 48.46, 53.85, 59.23, 64.61,
       70.00, 75.38, 80.77, 61.02, 62.82, 66.41, 68.20, 71.79, 55.64, 57.43]

In [144]:
x = []
y = []
z = []
for g,h in enumerate(points_and_coord):
    x.append(Udc[int(points_and_coord[h][0])])
    y.append(Urf[int(points_and_coord[h][1])])
    z.append(detect_rate[g])
    
fname = '2D_GMol_var_fluo_contour'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111)
title(all_subdir[0][:-15],fontsize=25)

im = ax.scatter(x,y,
        marker='o',s=400,edgecolor='black',zorder = 1000,
        c = z, cmap='rainbow')
ax.set_xlabel(r'$U_{DC}$ [V]')
ax.set_ylabel(r'$U_{RF}$ [V]')
ax.grid()

for i, txt in enumerate(points_and_coord):
    ax.annotate(str(i), (x[i], y[i]),color='xkcd:black',fontsize=13,
                ha='center',va='center',weight='bold',zorder = 1000)
    # ax.annotate(str(txt), (x[i], y[i]-0.70),fontsize=9)
#    if mean(cool_cloud[i])*13 < 13:
#        ax.annotate(f'{mean(cool_cloud[i])*13:.0f}',(x[i]-0.12, y[i]+0.25),fontsize=13,color='red',ha='right')
    #ax.annotate(str(txt)+'\n'+str(i), (x[i], y[i]))

cbar = plt.colorbar(im, ax=ax)
cbar.ax.set_ylabel(r'Detection rate (SNR > '+str(SNR_trig_threshold)+ ', Detuning -'+ str(Detuning) +   ')\n ('+str(len(num_runs))+' runs for each point)', rotation=90)
# ax.grid()
ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=1))
fig.set_size_inches(11.69, 8.27)

fig.tight_layout()
savefig(fname+'_D'+str(Detuning)+'.png',dpi=300)